# UTSA CS 3793/5233: Assignment-1

**Anderson - Kyle - jvh640**






## Learning Objectives


*   Read data from a file and Create a graph
*   Implement Uninformed & Informed searching strategies
*   Apply different searching strategies for the given problem
*   Analyze and Compare the searching strategies


## Description

This assignment is focused on **python file reading, graph creation** and implementation of **search algorithms**.
In the following sections, you will complete a series of tasks for a made up problem of *Coronavirus in Texas*.

*   Coronavirus is non-discriminatory, in the sense that it can spread from one city to any other city. The only goal of the virus is to spread to all cities in Texas. Find a possible way for the virus to spread (Uninformed Search).
*   To counter the effect of the virus, vaccine needs to be distributed to all cities. One city has more demand than supply, whereas one city has a shortage of vaccines. The goal is to find an **optimal** strategy to transport the vaccine (Informed Search) from the city with high supply (low demand) to the city with low supply (high demand).

The base structure and comments are provided on what should be done. You can use some libraries that help support you for the successful completion of the assignment. However, you **CANNOT** use a complete library for the search algorithms. You can get pieces of code from online, but please cite the source properly.


#Reading Data Files & Creating a 2D Graph

##(45 points)

In this section, you will write code to read the data files provided, cities.csv and distances.csv, and create a 2D graph consisting of nodes and edges. The goal is to use this graph for the 2 search agents that you will create in the next section.

Provided with this lab, on Blackboard, you will find 2 csv files:

*   **cities.csv** - This file contains a list of coordinates for selected cities in Texas, in the following format:
```
San Antonio,29.4685,-98.5254
```
The above line means that San Antonio is located at the latitude and longitude of 29.4685 N and 98.5254 W respectively. Note that the '-ve' sign denotes 'S' for latitude and 'W' for longitude. While performing calculations you will need to ignore the sign.

*   **distances.csv** - This file contains distance values between two cities of Texas, if a path exists, in the following format:
```
San Antonio,New Braunfels,30.80876734
```
The above line denotes that there should be an edge between *San Antonio* and *New Braunfels* and the weight on that edge, i.e. the distance, is *30.80876734*.

In the code blocks below, handle the logic for the graph. Load the graph data from the give files and display a 2D graph of the given data, with labeled nodes and edges. Create as many functions or code blocks as needed.

##Extra Credit (4 points)

Overlay the 2D graph on an image of the Texas state map.





In [ ]:
# Add only your imports here
import sys
import csv
import heapq
import math

class City:
    def __init__(self, city_name, latitude, longitude):
        self.__city_name = city_name
        self.__latitude = float(latitude)
        self.__longitude = float(longitude)
        self.__routes = []

    # Accessors
    def get_city_name(self):
        return self.__city_name

    def get_latitude(self):
        return self.__latitude

    def get_longitude(self):
        return self.__longitude

    def get_routes(self):
        return self.__routes

    # Mutators
    def set_city_name(self, city_name):
        self.__city_name = city_name

    def set_latitude(self, latitude):
        self.__latitude = latitude

    def set_longitude(self, longitude):
        self.__longitude = longitude

    def set_routes(self, routes):
        self.__routes = routes

    # Helper Methods
    def add_route(self, route):
        self.get_routes().append(route)

    def remove_route(self, route):
        self.get_routes().remove(route)

    def check_lat_pole(self):
        latitude = self.get_latitude()
        if latitude < 0:
            return f"{abs(latitude)} S"
        else:
            return f"{abs(latitude)} N"

    def check_long_pole(self):
        longitude = self.get_longitude()
        if longitude < 0:
            return f"{abs(longitude)} W"
        else:
            return f"{abs(longitude)} E"

    # String Override
    def __str__(self):
        ret = f"""{self.get_city_name()}, {self.check_lat_pole()}, {self.check_long_pole()}\nRoutes:\n"""
        for route in self.get_routes():
            ret += f"{route}\n"
        return ret

class Route:
    def __init__(self, city1, city2, distance):
        self.__cities = [city1, city2]
        self.__distance = float(distance)

    # Accessors
    def get_cities(self):
        return self.__cities

    def get_distance(self):
        return self.__distance

    # Mutators
    def set_cities(self, cities):
        self.__cities = cities

    def set_distance(self, distance):
        self.__distance = distance

    # Helper Methods
    def get_destination(self, current_city):
        if current_city == self.get_cities()[0]:
            return self.get_cities()[1]
        else:
            return self.get_cities()[0]

    # String Override
    def __str__(self):
        cities = self.get_cities()
        ret = f"   {cities[0].get_city_name()}, {cities[1].get_city_name()}, {self.get_distance()}"
        return ret

class Map:
    __instance = None

    def __init__(self):
        self.__state = "Texas"
        self.__cities = {}
        if Map.__instance is not None:
            raise Exception("Map is a singleton and can only be instantiated once")
        Map.__instance = self

    # Map Singleton
    @staticmethod
    def get_map_instance():
        if Map.__instance is None:
            Map.__instance = Map()
        return Map.__instance

    # Accessors
    def get_state(self):
        return self.__state

    def get_cities(self):
        return self.__cities

    # Mutators
    def set_state(self, state):
        self.__state = state

    def set_cities(self, cities):
        self.__cities = cities

    # Helper Methods
    # def uninformed_search(self):

    # def informed_search(self):

    def load_map(self, path):

        cities_file = f"{path}cities.csv"

        try:
            cities = self.get_cities()
            with open(cities_file) as csv_infile:
                cities_csv_reader = csv.reader(csv_infile)
                for row in cities_csv_reader:
                    tmp_city = City(row[0], row[1], row[2])
                    cities[tmp_city.get_city_name()] = tmp_city
        except FileNotFoundError:
            sys.stderr.write(f'''cities.csv not found at {cities_file}''')
            exit(1)

        distances_file = f"{path}distances.csv"
        try:
            with open(distances_file) as csv_infile:
                distances_csv_reader = csv.reader(csv_infile)
                for row in distances_csv_reader:
                    tmp_route = Route(cities[row[0]], cities[row[1]], row[2])
                    cities[row[0]].add_route(tmp_route)
                    cities[row[1]].add_route(tmp_route)
        except FileNotFoundError:
            sys.stderr.write(f'''distances.csv not found at {distances_file}''')
            exit(1)

    def uniform_cost_search(self, starting_city_name):
        starting_city = self.get_cities()[starting_city_name]

        queue = [(0, starting_city)]
        visited_cities = {starting_city: 0}
        infection_order = []

        while queue:
            curr_cost, curr_city = heapq.heappop(queue)
            infection_order.append(curr_city)

            for route in curr_city.get_routes():
                next_city = route.get_destination(curr_city)
                new_cost = curr_cost + route.get_distance()

                if next_city not in visited_cities or new_cost < visited_cities[next_city]:
                    visited_cities[next_city] = new_cost
                    heapq.heappush(queue, (new_cost, next_city))

        return infection_order

    def a_star_search(self, starting_city_name, destination_name):
        starting_city = self.get_cities()[starting_city_name]
        destination_city = self.get_cities()[destination_name]

        queue = [(0, starting_city)]
        visited_cities = {starting_city: 0}
        total_route = []

        while queue:
            curr_cost, curr_city = heapq.heappop(queue)
            total_route.append(curr_city)

            if curr_city == destination_city:
                return total_route, curr_cost

            for route in curr_city.get_routes():
                next_city = route.get_destination(curr_city)
                new_cost = visited_cities[curr_city] + route.get_distance()

                if next_city not in visited_cities or new_cost < visited_cities[next_city]:
                    visited_cities[next_city] = new_cost
                    score = new_cost + Map.heuristic_haversine(next_city, destination_city)
                    heapq.heappush(queue, (score, next_city))

        return None

    @staticmethod
    def heuristic_haversine(next_city, destination):
        from_lat = next_city.get_latitude()
        from_long = next_city.get_longitude()
        to_lat = destination.get_latitude()
        to_long = destination.get_longitude()

        earth_radius_miles = 3958.8

        # Convert Long and Lat to radians
        next_lat_rad = math.radians(from_lat)
        next_long_rad = math.radians(from_long)
        dest_lat_rad = math.radians(to_lat)
        dest_long_rad = math.radians(to_long)

        a = pow(math.sin((next_lat_rad - dest_lat_rad)/2), 2) + math.cos(dest_lat_rad) * math.cos(next_lat_rad) * pow(math.sin((next_long_rad - dest_long_rad) / 2), 2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return earth_radius_miles * c

    # String Override
    def __str__(self):
        cities = self.get_cities()
        ret = f"Map of {self.get_state()}\n------------------------------------------------------------\n"
        for city in cities:
            ret += f"{cities[city]}\n"
        return ret


In [ ]:
# Assume that the data files are in the following folder -- THIS WILL BE USED BY THE TA
basePath = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/"


In [ ]:
# Load the graph data from the files
texas_map = Map.get_map_instance()
texas_map.load_map(basePath)
print(texas_map)

Map of Texas
------------------------------------------------------------
Abilene, 32.4543 N, 99.7384 W
Routes:
   San Angelo, Abilene, 95.26906977

Alice, 27.7556 N, 98.0653 W
Routes:
   Alice, Laredo, 98.54431104
   Alice, Three Rivers, 51.26861733
   Alice, McAllen, 113.6156404

Amarillo, 35.1989 N, 101.831 W
Routes:
   Lubbock, Amarillo, 122.9827931
   Amarillo, Dalhart, 85.07238807

Austin, 30.3006 N, 97.7517 W
Routes:
   Austin, San Marcos, 30.71742595
   Austin, Round Rock, 18.46400955
   Austin, College Station, 106.7588224
   Austin, Houston, 165.9076541

Beaumont, 30.085 N, 94.1451 W
Routes:
   Houston, Beaumont, 99.57951346

Brownsville, 25.998 N, 97.4565 W
Routes:
   McAllen, Brownsville, 59.20599771

College Station, 30.5852 N, 96.296 W
Routes:
   Austin, College Station, 106.7588224
   College Station, Waco, 97.82787143
   College Station, Houston, 99.20917694

Columbus, 29.7055 N, 96.5563 W
Routes:
   Houston, Columbus, 72.51512174
   Columbus, Seguin, 89.60403948

Corpu

In [ ]:
# Display a 2D graph of the given data.


#Virus Spread - Uninformed Search Agent

##(40 points)

In this section, you will use the graph created in the previous section and create an *uninformed search* agent that will print the path how the virus will spread to all the provided Texas cities. The first confirmed case of the virus was in **Three Rivers** and starts spreading from there. The virus does not discriminate and it needs to spread to all the cities of Texas.

In the following code block, write code to implement **any** uninformed search strategy. You are free to create more code blocks if needed. As the output, print

*   The path or sequence of cities that will be infected by the spread of Coronavirus.
*   The distance travelled by the selected virus spreading strategy.

##Extra Credit (3 points)
On the 2D graph and the Texas state map, overlay the selected path along with the cities visited.

In [ ]:
# Implement ANY uninformed search strategy for the spread of coronavirus from the starting city of 'Three Rivers'
# Uniform Cost Search - Located in Map Class
uninformed_infection_order = texas_map.uniform_cost_search("Three Rivers")
ret = "Uninformed Infection Order - Uniform Cost Search\n------------------------------------\n"
i = 1
for city in uninformed_infection_order:
    ret += f"{i}. {city.get_city_name()}\n"
    i += 1
print(ret)


Uninformed Infection Order - Uniform Cost Search
------------------------------------
1. Three Rivers
2. Kenedy
3. Alice
4. San Antonio
5. Corpus Christi
6. New Braunfels
7. Seguin
8. San Marcos
9. Gonzalez
10. Laredo
11. Austin
12. Uvalde
13. McAllen
14. Victoria
15. Round Rock
16. Columbus
17. Temple
18. Brownsville
19. Del Rio
20. Waco
21. College Station
22. Houston
23. San Angelo
24. Houston
25. Sugar Land
26. Galveston
27. Jamaica Beach
28. Dallas
29. Fort Worth
30. Beaumont
31. Palestine
32. Abilene
33. Midland
34. Odessa
35. Lubbock
36. Wichita Falls
37. Texarkana
38. Amarillo
39. Dalhart
40. El Paso



#Vaccine Transportation - Informed Search Agent

##(40 points)

In this section, you will create an *informed search* agent that will be used to transport the vaccine. The city of **San Antonio** has more supply of vaccine than the demand. The goal is to create an **optimal strategy** to transport the vaccine and make it available at the highly affected city of **College Station**, where there is a shortage of vaccines.

In the following code block, write code to implement an **optimal** informed search strategy. You are free to create more code blocks if needed. As the output, print

*   The path / sequence of cities that will be visited in the optimal vaccine transportation strategy.
*   The total distance travelled in the optimal vaccine transportation strategy.


##Extra Credit (3 points)
On the 2D graph and the Texas state map, overlay the selected path along with the cities visited.

In [ ]:
# Implement an OPTIMAL informed search strategy for distributing the vaccine from 'San Antonio' to 'College Station'
informed_route_order, distance = texas_map.a_star_search("San Antonio", "College Station")
ret = "Informed Route Order - A* Search\n------------------------------------\n"
i = 1
for city in informed_route_order:
    ret += f"{i}. {city.get_city_name()}\n"
    i += 1
ret += f"Total Distance: {distance}"
print(ret)


Informed Route Order - A* Search
------------------------------------
1. San Antonio
2. Seguin
3. New Braunfels
4. San Marcos
5. Gonzalez
6. Austin
7. Round Rock
8. Columbus
9. College Station
Total Distance: 186.77450141


#Submission Instructions

1.   Complete all tasks above - **File MUST contain the output for ALL cells**
2.   Export this notebook as .ipynb
      (File > Download as ipynb)
3.   Upload the .ipynb file on Blackboard

##Rubric

*   (45 points) Reading Data files & Creating a 2D Graph
*   (40 points) Virus Spread - Uninformed Search Agent
*   (40 points) Vaccine Transportation - Informed Search Agent
*   (10 points) Extra Credit - on the Texas map image, overlay the 2D graph and the paths selected by the search agents



